In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.quant = torch.ao.quantization.QuantStub()
        self.model = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(8, 8*8, 3, stride=2),
            nn.ReLU(),
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10) 
            # nn.Softmax()
        )
        self.dequant = torch.ao.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        y = self.model(x)
        return self.dequant(y)
    

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to("cuda")
        y = y.to("cuda")
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
model = Classifier()

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
loss_fn = nn.CrossEntropyLoss()

In [4]:
model.to("cuda")
model.eval()
backend = "x86"
model.qconfig = torch.quantization.get_default_qat_qconfig(backend)

model_fp32_prepared = torch.quantization.prepare_qat(model.train(), inplace=False)

for i in range(200):
    train(train_dataloader, model_fp32_prepared, loss_fn, optimizer)

/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/fake_quantize.py:353: UserWarning: _aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This warning will only appear once per process. (Triggered internally at ../aten/src/ATen/native/ReduceAllOps.cpp:71.)
  return torch.fused_moving_avg_obs_fake_quant(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/fake_quantize.py:353: UserWarning: _aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This warning will only appear once per process. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:687.)
  retur

loss: 2.302333  [   64/60000]
loss: 2.287039  [ 6464/60000]
loss: 2.297430  [12864/60000]
loss: 2.303822  [19264/60000]
loss: 2.301533  [25664/60000]
loss: 2.309207  [32064/60000]
loss: 2.322660  [38464/60000]
loss: 2.307912  [44864/60000]
loss: 2.309490  [51264/60000]
loss: 2.306941  [57664/60000]
loss: 2.298245  [   64/60000]
loss: 2.289940  [ 6464/60000]
loss: 2.298312  [12864/60000]
loss: 2.306079  [19264/60000]
loss: 2.311275  [25664/60000]
loss: 2.306083  [32064/60000]
loss: 2.300441  [38464/60000]
loss: 2.304611  [44864/60000]
loss: 2.313630  [51264/60000]
loss: 2.313148  [57664/60000]
loss: 2.294288  [   64/60000]
loss: 2.309016  [ 6464/60000]
loss: 2.305173  [12864/60000]
loss: 2.292884  [19264/60000]
loss: 2.299614  [25664/60000]
loss: 2.297619  [32064/60000]
loss: 2.306399  [38464/60000]
loss: 2.298535  [44864/60000]
loss: 2.308318  [51264/60000]
loss: 2.315697  [57664/60000]
loss: 2.293626  [   64/60000]
loss: 2.299409  [ 6464/60000]
loss: 2.292240  [12864/60000]
loss: 2.29

In [5]:
model_fp32_prepared.to("cpu")
model_fp32_prepared.eval()
model_int8 = torch.quantization.convert(model_fp32_prepared, inplace=False)

In [6]:
%%timeit
test(test_dataloader, model_fp32_prepared, loss_fn)

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303754 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303747 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303748 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303743 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303737 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303732 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303733 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303732 

457 ms ± 2.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
test(test_dataloader, model_int8, loss_fn)

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303760 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303760 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303760 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303760 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303760 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303760 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303760 

Test Error: 
 Accuracy: 9.9%, Avg loss: 2.303760 

690 ms ± 290 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
torch.save(model_int8, "model_int8.pt")